# Flow Over a Parameterized Block Example Guide

## Overview
This guide explains how to implement a 2D flow over a parameterized block (chip) using PhysicsNeMo. The example demonstrates how to set up a Navier-Stokes solver for a channel with a rectangular obstacle (chip) and how to apply various boundary and interior constraints, as well as validation against reference data.
<center><img src="images/chip_2d.png" alt="Drawing" style="width:500px" /></center>

## Problem Description
- **Domain:** 2D channel with a rectangular chip (block) inside
- **Physics:** Steady-state incompressible Navier-Stokes equations
- **Goal:** Predict the velocity and pressure fields around the chip

Equation to solve: Navier Stokes (continuity, momentum-x, momentum-y)
## Mathematical Formulation

$$\begin{align*} \text{continuity: }\quad\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}  = 0\\
\text{momentum-x: }\quad u\frac{\partial u}{\partial x} + v\frac{\partial u}{\partial y} +\frac{1}{\rho}\frac{\partial p}{\partial x} - \nu \frac{\partial^2 u}{\partial x^2} - \nu \frac{\partial^2 u}{\partial y^2} = 0 \\
\text{momentum-y: } \quad u\frac{\partial v}{\partial x} + v\frac{\partial v}{\partial y} +\frac{1}{\rho}\frac{\partial p}{\partial y} - \nu \frac{\partial^2 v}{\partial x^2} - \nu \frac{\partial^2 v}{\partial y^2} = 0 \\
\end{align*}
$$

where:
- Boundary conditions: Parabolic inlet velocity, zero outlet pressure, no slip walls
- Fluid density: $\rho = 1.0$, fluid viscosity: $\nu = 0.02$


## Implementation Steps

### 1. Import Required Libraries
```python
import os
import warnings

import numpy as np
from sympy import Symbol, Eq, And, Or

import physicsnemo.sym
from physicsnemo.sym.hydra import to_absolute_path, instantiate_arch, PhysicsNeMoConfig
from physicsnemo.sym.utils.io import csv_to_dict
from physicsnemo.sym.solver import Solver
from physicsnemo.sym.domain import Domain
from physicsnemo.sym.geometry.primitives_2d import Rectangle, Line, Channel2D
from physicsnemo.sym.utils.sympy.functions import parabola
from physicsnemo.sym.eq.pdes.navier_stokes import NavierStokes
from physicsnemo.sym.eq.pdes.basic import NormalDotVec
from physicsnemo.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)

from physicsnemo.sym.domain.validator import PointwiseValidator
from physicsnemo.sym.utils.io import ValidatorPlotter
from physicsnemo.sym.key import Key
from physicsnemo.sym.node import Node

import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'jet'
```

### 2. Define the Main Function and Physics
- Use the `@physicsnemo.sym.main` decorator for configuration.
- Instantiate the 2D steady state Navier-Stokes equations.
- Instantiate neural network architecture (fully-connected)
- Create PhysicsNeMo nodes

```python
@physicsnemo.sym.main(config_path="conf", config_name="config_chip_2d")
def run(cfg: PhysicsNeMoConfig) -> None:
    ns = NavierStokes() # Define the Navier-Stokes equations
    normal_dot_vel = NormalDotVec(["u", "v"])
    flow_net = instantiate_arch(
        input_keys=[Key("x"), Key("y")],
        output_keys=[Key("u"), Key("v"), Key("p")],
        cfg=cfg.arch.fully_connected,
    )
    nodes = (
        ns.make_nodes()
        + normal_dot_vel.make_nodes()
        + [flow_net.make_node(name="flow_network")]
    )
```

### 3. Define Geometry and Domain
- Define the geometry parameters and ranges 
- Define the inlet, outlet, channel, and block 
- Create the geometry by subtracting block from the channel 
    - Note that channel and rectangle geometry objects are different. When sampling from the boundaries, no points will be samples from the two ends of a channel.
- Define the parameterized integral line
    - This will be used to enforce conservation of total flow through random Integral continuity planes 

```python
    channel_length = (-2.5, 2.5)
    channel_width = (-0.5, 0.5)
    chip_pos = -1.0
    chip_height = 0.6
    chip_width = 1.0
    inlet_vel = 1.5

    x, y = Symbol("x"), Symbol("y")
    channel = Channel2D((channel_length[0], channel_width[0]), (channel_length[1], channel_width[1]))
    rec = Rectangle((chip_pos, channel_width[0]), (chip_pos + chip_width, channel_width[0] + chip_height))
    geo = channel - rec
    
    inlet = Line((channel_length[0], channel_width[0]), (channel_length[0], channel_width[1]), normal=1)
    outlet = Line((channel_length[1], channel_width[0]), (channel_length[1], channel_width[1]), normal=1)
    x_pos = Symbol("x_pos")
    integral_line = Line((x_pos, channel_width[0]), (x_pos, channel_width[1]), 1)
    x_pos_range = {x_pos: lambda batch_size: np.full((batch_size, 1), np.random.uniform(channel_length[0], channel_length[1]))}

    domain = Domain()
```

### 4. Add Constraints
- Define the inlet constraint
    - Parabolic u-velocity, zero v-velocity
- Define the outlet constraint 
    - Zero pressure
- Define the wall constraint
    - No-slip walls: zero u-velocity and v-velocity
#### Inlet (Parabolic Profile)
```python
    inlet_parabola = parabola(y, channel_width[0], channel_width[1], inlet_vel)
    inlet = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=inlet,
        outvar={}, #inlet boundary condition 
        batch_size=cfg.batch_size.inlet,
    )
    domain.add_constraint(inlet, "inlet")
```

#### Outlet (Pressure)
```python
    outlet = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=outlet,
        outvar={}, #outlet boundary condition 
        batch_size=cfg.batch_size.outlet,
        criteria=Eq(x, channel_length[1]),
    )
    domain.add_constraint(outlet, "outlet")
```

#### No-Slip (Walls and Chip)
```python
    no_slip = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={}, #no slip boundary condition 
        batch_size=cfg.batch_size.no_slip,
    )
    domain.add_constraint(no_slip, "no_slip")
```

#### Interior Constraints (Physics)
- Define the interior constraint
    - Conservation of continuity and momentum 
- Use the signed distance function (SDF) of the interior to mitigate the difficulty of learning sharp local gradients 
    - This will weight losses lower on sharp gradients or discontinuous areas of the domain
- Signed distance function is defined as
$$ \begin{align*} S(x) =  \begin{cases} d(x,\partial D), \quad x \in D \\ -d(x,\partial D), \quad x \in D^c  \end{cases}\end{align*}$$
where d is the distance function, and D is domain.
The below figure shows a batch of interior point cloud , color coded with SDF . 
SDF is used for weighting the continuity and momentum equations for better convergence
<center><img src="images/chip_2d_sdf.png" alt="Drawing" style="width:500px" /></center

```python
    interior = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={}, #PDE constraint
        batch_size=cfg.batch_size.interior,
        lambda_weighting={
            "continuity": 2 * Symbol("sdf"),
            "momentum_x": 2 * Symbol("sdf"),
            "momentum_y": 2 * Symbol("sdf"),
        },
    )
    domain.add_constraint(interior, "interior")
```

#### Integral Continuity Constraint
- In addition to solving the NS equations in differential form, specifying the volumetric flow rate through some integral continuity lines or planes that are located across the channel significantly speeds up the convergence rate and improves accuracy.
- Define a callable for sampling criteria
- Define the integral constraint 
    - At each training iteration, this will create multiple random lines throughout the channel for which the total flow is computed, and adds a term to the loss function to enforce the total flow passing through each of those lines to be equal to the total inflow 
    - Integral_batch_size: Number of points within each line, to be used for Monte Carlo approximation of total flow 
    - batch_size: Number of random lines at each iteration 

```python
    def integral_criteria(invar, params):
        sdf = geo.sdf(invar, params)
        return np.greater(sdf["sdf"], 0)

    integral_continuity = IntegralBoundaryConstraint(
        nodes=nodes,
        geometry=integral_line,
        outvar={"normal_dot_vel": 1},
        batch_size=cfg.batch_size.num_integral_continuity,
        integral_batch_size=cfg.batch_size.integral_continuity,
        lambda_weighting={"normal_dot_vel": 1},
        criteria=integral_criteria,
        parameterization=x_pos_range,
    )
    domain.add_constraint(integral_continuity, "integral_continuity")
```

### 5. Add Validation
- Import and process the validation data
    - Specify a proper mapping 
    - Perform the required transformations to the data to match units, scales 
    - Define a dictionary of input and output variables for the validator
- Define the validator

```python
    file_path = "examples_sym/examples/chip_2d/openfoam/2D_chip_fluid0.csv"
    if os.path.exists(to_absolute_path(file_path)):
        mapping = {"Points:0": "x", "Points:1": "y", "U:0": "u", "U:1": "v", "p": "p"}
        openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
        openfoam_var["x"] -= 2.5  # normalize pos
        openfoam_var["y"] -= 0.5
        openfoam_invar_numpy = {key: value for key, value in openfoam_var.items() if key in ["x", "y"]}
        openfoam_outvar_numpy = {key: value for key, value in openfoam_var.items() if key in ["u", "v", "p"]}
        openfoam_validator = PointwiseValidator(
            nodes=nodes,
            invar=openfoam_invar_numpy,
            true_outvar=openfoam_outvar_numpy,
            plotter=ValidatorPlotter(),
        )
        domain.add_validator(openfoam_validator)
    else:
        warnings.warn(
            f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/physicsnemo/resources/physicsnemo_sym_examples_supplemental_materials"
        )
```

### 6. Create and Run the Solver
```python
    slv = Solver(cfg, domain)
    slv.solve()
```

## Key Components Explained

1. **Navier-Stokes Physics Class**:
   - Defines the mathematical formulation of the 2D steady-state Navier-Stokes equations
   - Uses SymPy for symbolic computation
   - Handles viscosity, density, and incompressibility

2. **Neural Network**:
   - Takes x and y as inputs
   - Outputs velocity components (u, v) and pressure (p)
   - Uses a fully connected architecture

3. **Constraints**:
   - Inlet: Parabolic velocity profile at the channel entrance
   - Outlet: Pressure boundary at the channel exit
   - No-slip: Zero velocity on channel walls and chip surface
   - Interior: Enforces the Navier-Stokes equations throughout the domain, with higher resolution near the chip
   - Integral: Enforces mass conservation across the channel

4. **Validation**:
   - Uses reference data (e.g., OpenFOAM) for validation if available
   - Compares neural network predictions with reference solution
   - Visualizes results using matplotlib

## Running the Example

1. Fill in all missing parts
2. Place the configuration file in the conf directory
3. Run the script:
```bash
python chip_2d.py
```

## Expected Results
The solver will:
1. Train the neural network to solve the flow over the parameterized block
2. Generate validation plots comparing the solution with the reference data